# Assignment

In this assignment we want to get comfortable with loading and manipulating data in Python. While future assignments will focus more using structured data which we can load into a `DataFrame` using `pandas`, this assignment is focused on semi-structured data and how we can "flatten" it and then load it into other formats. The objective is to see how data flows in Python from one object to another and what advantages and disadvantages each offers.

Let's read the `books.json` data set and display the first item in it.

In [84]:
import json
with open('data/books.json', encoding = 'utf-8') as f:
    books_dict = json.load(f)

from pprint import pprint
pprint(books_dict[0]) # print information for the first book

{'_id': 1,
 'authors': ['W. Frank Ableson', 'Charlie Collins', 'Robi Sen'],
 'categories': ['Open Source', 'Mobile'],
 'isbn': '1933988673',
 'longDescription': 'Android is an open source mobile phone platform based on '
                    'the Linux operating system and developed by the Open '
                    'Handset Alliance, a consortium of over 30 hardware, '
                    'software and telecom companies that focus on open '
                    'standards for mobile devices. Led by search giant, '
                    'Google, Android is designed to deliver a better and more '
                    'open and cost effective mobile experience.    Unlocking '
                    "Android: A Developer's Guide provides concise, hands-on "
                    'instruction for the Android operating system and '
                    'development tools. This book teaches important '
                    'architectural concepts in a straightforward writing style '
                    

1. Write a program that goes through the entire data and extracts the following information:  <span style="color:red" float:right>[4 point]</span>

  - title of the book
  - name of the first author
  - name of the second author (if book has more than one author)
  - number of authors
  - ISBN
  - if the word "data" is in the book's description
  - the number of words in the book's description
  - the year the book was published

  Of course because JSON data doesn't necessarily enforce any sort of schema, we can't be sure that the information we are trying to extract exists for every book. For example, if the book only has one author, then there is no second author. So use `try` and `except` as you loop through every book and skip to the next item every time some information is missing.

  Store the extracted data in a list named `rows` whose elements are tuples, one tuple per book. For example, the first element of `rows` stores the tuple for the first book and should look like this:

        ('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', True, 252, 2009)

In [86]:
rows = []
for i in books_dict: # loops through "books"
    # adds title, all books have  
    b = [i['title']] 
    
    try: #checks for author 1 and adds
        b = b + [i['authors'][0]]
    except:
        b = b + [None]
    
    try: #checks for author 2 and adds
        b = b + [i['authors'][1]]
    except:
        b = b + [None]
        
    # adds num of authors    
    b = b + [len(i['authors'])]
    
    try: #checks for isbn and adds
        b = b + [i['isbn']]
    except:
        b = b + [None]
    
    try: #checks for portions about description and adds
        b = b  + ['data'in i['longDescription']] + [len(i['longDescription'].split())]
    except:
        b = b + [None] + [None]
    
    try: #checks for date and adds
        b = b + [int(str(i['publishedDate'])[11:15])]
    except:
        b = b + [None]
    rows.append(tuple(b)) # adds row to rows as tuple


2. Save the content of `rows` in a SQL-like table using `sqlite3`, and choose the appropriate column types. <span style="color:red" float:right>[2 point]</span> 

  As your column names use the following:

  - `title`
  - `author_1`
  - `author_2`
  - `num_authors`
  - `isbn`
  - `has_data`
  - `desc_len`
  - `year_published`

In [87]:
import sqlite3

# establish connection and cursor
connection = sqlite3.connect(':memory:') 
cursor = connection.cursor()

# creates table from rows with datatypes
cursor.execute('''CREATE TABLE books_long (
                            title TEXT,
                            author_1 TEXT, 
                            author_2 TEXT,
                            num_authors TINYINT,
                            isbn TEXT,
                            has_data BOOLEAN,
                            desc_len SMALLINT,
                            year_published SMALLINT 
                )''')

cursor.executemany('INSERT INTO books_long VALUES (?,?,?,?,?,?,?,?)', rows)
connection.commit() # save the changes

3. Write a SQL query against the table to show all books that (1) contain the word "data" and (2) have more than 3 authors. Store the result of the query in an object called `books_table`, then close the connection. <span style="color:red" float:right>[2 point]</span>

In [88]:
# makes query and finds results that match 
query =  '''SELECT * FROM books_long WHERE (has_data = True) AND (num_authors > 3) '''
books_table = cursor.execute(query).fetchall()

#closes sqlite tools
cursor.close()
connection.close()

SQL tables are not the only way, and definitely not the most straightforward way to store and manipulate data in Python. A format that's more popular with data scientist is to use the `pandas` library to create a `DataFrame`. This library has a lot of functionality that makes it easy to run the common tasks data scientists do with data.

4. Read the data from the above query into a `DataFrame` and call it `books_df`. HINT: Use `pd.DataFrame` and specify the names to use for the columns. <span style="color:red" float:right>[1 point]</span>

In [89]:
import pandas as pd
#creates data frame 
books_df = pd.DataFrame(books_table,columns = ['title', 'author_1', 'author_2','num_authors', 'isbn', 'has_data', 'desc_len', 'year_published'])

5. Display the first few columns of a `DataFrame` by calling its `head` method. <span style="color:red" float:right>[1 point]</span>

In [90]:
#prints head
books_df.head()

,title,author_1,author_2,num_authors,isbn,has_data,desc_len,year_published
0,Flex 4 in Action,Tariq Ahmed,Dan Orlando,4,1935182420,1,329,2010.0
1,Spring Dynamic Modules in Action,Arnaud Cogoluegnes,Thierry Templier,4,1935182307,1,183,2010.0
2,Android in Practice,Charlie Collins,Michael D. Galpin,4,1935182927,1,196,2011.0
3,Portlets and Apache Portals,Stefan Hepper,Peter Fischer,5,None,1,153,2005.0
4,Struts in Action,Ted N. Husted,Cedric Dumoulin,4,1932394249,1,137,2002.0


Remember how earlier we said that a `DataFrame` is built on top of `numpy` arrays? Another way of saying it is that a `DataFrame` is an **abstraction** on top of `numpy` arrays: i.e. a `DataFrame` is a more **high-level** object than a `numpy` array. 

6. Call the `values` attribute of your `DataFrame` to convert it into a numpy array and display the first 3 elements of the array. <span style="color:red" float:right>[1 point]</span>

In [56]:
#converts DF to NParray and prints 
np_books = books_df.values
print(np_books[0:3])

[['Flex 4 in Action' 'Tariq Ahmed' 'Dan Orlando' 4 '1935182420' 1 329
  None]
 ['Spring Dynamic Modules in Action' 'Arnaud Cogoluegnes'
  'Thierry Templier' 4 '1935182307' 1 183 None]
 ['Android in Practice' 'Charlie Collins' 'Michael D. Galpin' 4
  '1935182927' 1 196 None]]


Now you can judge which object is more "user-friendly". That's one of the things that abstractions allow us to do: build more user-friendly (abstract) objects from less user-friendly (but more fundamental) objects.

Brian: it is pretty obvious that the Pandas Data Frame much more user-friendly than the numpy array.

# End of assignment